In [8]:
import training
import numpy as np
import regularizers

import os

Experiments involving the mazegen task from the seRNNs paper
======

Generate mazegen data

In [ ]:
import mazegen

gen = mazegen.mazeGeneratorI(goal_presentation_steps=20,delay_steps=10,choices_presentation_steps=20)
tfdf = gen.construct_tf_data(number_of_problems=5120, batch_size=128)
tfdf_test = gen.construct_tf_data(number_of_problems=2560, batch_size=128)

Define regularization strengths

In [10]:
num_reg_strengths = 5
strengths = np.linspace(0.0, 1.0, num_reg_strengths)

Train 50 L1-Regularized RNNs with regularization strength between 0 and 1

In [11]:
L1_regus = [regularizers.L1(st) for st in strengths]
histories = training.train(L1_regus, tfdf, tfdf_test)
training.save_histories(histories, "mazegen/L1")

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5489 - loss: 1.0443 - val_accuracy: 0.7574 - val_loss: 0.3676
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7701 - loss: 0.4155 - val_accuracy: 0.8836 - val_loss: 0.2559
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8801 - loss: 0.2316 - val_accuracy: 1.0000 - val_loss: 0.0220
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0086 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 4.3827e-04
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 4.3065e-04 - val_accuracy: 1.0000 - val_loss: 2.7162e-04
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 2.7544e-04 - val_accuracy: 1.0000 - val_loss: 1.8959e-04
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 2.0140e-04 - 

FileNotFoundError: [Errno 2] No such file or directory: 'data/mazegen/L1/history.pickle'